# 資料處理


---
將現有的圖片資料庫擴充後再轉換成各個像素RGB值的csv檔，並且帶有label



# 導入資料

讀取照片資料夾

In [ ]:
# 掛載google雲端
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 檢查雲端內容
!ls "/content/drive/My Drive"

'Benign keratosis'   C64111174_王宇桐_D組.gdoc	'Colab Notebooks'   模型data


In [ ]:
# 資料夾路徑
images_path = '/content/drive/My Drive/模型data'

# 擴增資料庫


In [ ]:
!pip install pillow opencv-python

將資料夾中的每張圖片旋轉、變形後再調亮度最後模糊化，原本70張照片旋轉後變280張，再來變形後變560張，再來調亮度變1120張，最後模糊化變2240張。

將照片轉為RGB模式

In [ ]:
from PIL import Image
import os

# 定義圖片處理函數：將圖片轉換為 RGB 模式
def convert_to_rgb(image_path):
    image = Image.open(image_path)
    if image.mode != 'RGB':
        image = image.convert('RGB')
        image.save(image_path)

# 遍歷資料夾中的每張圖片並轉換為 RGB 模式
for filename in os.listdir(images_path):
    if filename.endswith(('.png', '.jpg', '.jpeg')): # 依照圖片格式改變
        img_path = os.path.join(images_path, filename)
        convert_to_rgb(img_path)

print("資料集中所有照片已成功轉換為 RGB 模式。")

資料集中所有照片已成功轉換為 RGB 模式。


**照片旋轉**

In [ ]:
# 照片資料夾中每一張照片都分別旋轉90 180 270度變成新的一張後再儲存進資料夾中，資料夾的照片變四倍
# 會跑
from PIL import Image
import os

for filename in os.listdir(images_path):
    if filename.endswith(('.png', '.jpg', '.jpeg')):  # 依照圖片格式改變
        img_path = os.path.join(images_path, filename)

        # 讀取圖片
        img = Image.open(img_path)

        # 對每張圖片進行90, 180, 270度旋轉
        for angle in [90, 180, 270]:
            rotated_img = img.rotate(angle)

            # 新的文件名和路徑
            new_filename = f'{os.path.splitext(filename)[0]}_{angle}.jpg'
            new_img_path = os.path.join(images_path, new_filename)

            rotated_img.save(new_img_path) # 儲存旋轉完的照片

print("圖片處理完成。")


圖片處理完成。


**照片變形**

In [ ]:
!pip install imgaug Pillow

In [ ]:
# 將資料夾中每一張照片都變形後變成一張新的照片儲存進資料夾中，資料夾的照片變兩倍
import imgaug.augmenters as iaa
import numpy as np


# 圖片變形函數
def apply_image_distortion(image_array):
    seq = iaa.Sequential([
        iaa.Affine(translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)}, rotate=(-10, 10))
    ])
    distorted_image = seq(image = image_array)
    return distorted_image

# 遍歷資料夾中的每張圖片
for filename in os.listdir(images_path):
    if filename.endswith(('.png', '.jpg', '.jpeg')):  # 依照圖片格式改變
        img_path = os.path.join(images_path, filename)

        # 讀取圖片並轉換為numpy array
        img = Image.open(img_path)
        img_array = np.array(img)

        # 應用圖像變形
        distorted_img_array = apply_image_distortion(img_array)

        # 確保numpy array是uint8類型
        distorted_img_array = np.clip(distorted_img_array, 0, 255).astype(np.uint8)

        # 將numpy array轉換回PIL圖像
        distorted_img = Image.fromarray(distorted_img_array)

        # 新的文件名和路徑
        new_filename = f'distorted_{filename}'
        new_img_path = os.path.join(images_path, new_filename)

        distorted_img.save(new_img_path) # 儲存變形完的照片

print("圖片變形處理完成。")


圖片變形處理完成。


**照片變暗**

In [ ]:
# 將資料夾中每一張照片都變暗變成一張新的照片儲存進資料夾中，資料夾照片變兩倍
from PIL import ImageEnhance, Image

# 定義函數：將圖片變暗
def darken_image(image):
    enhancer = ImageEnhance.Brightness(image)
    darkened_image = enhancer.enhance(0.5)  # 0.5 表示將亮度降低為原始的一半
    return darkened_image

# 遍歷資料夾中的每張圖片
for filename in os.listdir(images_path):
    if filename.endswith(('.png', '.jpg', '.jpeg')):  # 可以根據需要添加其他圖片格式
        img_path = os.path.join(images_path, filename)

        # 讀取圖片
        img = Image.open(img_path)

        # 將圖片變暗
        darkened_img = darken_image(img)

        # 構建新的文件名和路徑
        new_filename = f'darkened_{filename}'
        new_img_path = os.path.join(images_path, new_filename)

        # 保存變暗後的圖片
        darkened_img.save(new_img_path)

print("圖片變暗處理完成。")


圖片變暗處理完成。


**照片模糊化**

In [ ]:
# 將資料夾中每一張照片都模糊化變成一張新的照片儲存進資料夾中，資料夾照片變兩倍
from PIL import ImageFilter

# 定義函數：應用高斯模糊
def apply_gaussian_blur(image):
    sigma = np.random.uniform(1, 3)  # 隨機選擇標準差以增加多樣性
    blurred_image = image.filter(ImageFilter.GaussianBlur(sigma))
    return blurred_image

# 遍歷資料夾中的每張圖片
for filename in os.listdir(images_path):
    if filename.endswith(('.png', '.jpg', '.jpeg')):  # 可以根據需要添加其他圖片格式
        img_path = os.path.join(images_path, filename)

        # 讀取圖片
        img = Image.open(img_path)

        # 應用高斯模糊
        blurred_img = apply_gaussian_blur(img)

        # 構建新的文件名和路徑
        new_filename = f'blurred_{filename}'
        new_img_path = os.path.join(images_path, new_filename)

        # 保存模糊化後的圖片
        blurred_img.save(new_img_path)

print("圖片模糊化處理完成。")


圖片模糊化處理完成。


**得到一個新的資料集**


In [ ]:
# 將照片資料集做處理，調成28X28
# 設定新的照片大小
new_size = (28, 28)

# 遍歷資料夾中的每張圖片
for idx, filename in enumerate(os.listdir(images_path)):
    if filename.endswith(('.png', '.jpg', '.jpeg')):  # 可以根據需要添加其他圖片格式
        img_path = os.path.join(images_path, filename)

        img_path = os.path.join(images_path, filename)

        # 讀取圖片
        img = Image.open(img_path)

        # 格式化為新的大小
        img_resized = img.resize(new_size)

        # 覆盖原始圖片
        img_resized.save(img_path)

print("圖片格式化處理完成。")

圖片格式化處理完成。


# 轉換照片變成各個像素位置的RGB三個值

將資料集中每個圖片轉換成像素RGB值的csv檔，是一個陣列，總共會有28X28X3個像素值，因為每個像素值都有RGB三個值，所以csv檔總共有2353行，分別是每一個像素的RGB三值，最後則是標籤

In [25]:
import os
from PIL import Image
import pandas as pd
import numpy as np

# 初始化Dataframe來儲存數據
df = pd.DataFrame(columns=['pixel{:04d}'.format(i) for i in range(28 * 28 * 3)] + ['label'])

# 遍歷每張照片
for filename in os.listdir(images_path):
    if filename.endswith(('.png', '.jpg', '.jpeg')):  # 根據照片格式
        img_path = os.path.join(images_path, filename)

        # 讀取圖片
        image = Image.open(img_path)

        # 將圖片轉為RGB值的numpy數據
        image_array = np.array(image)

        # 將數據連接到一起並添加到DataFrame中
        flattened_image_array = image_array.flatten()
        df_temp = pd.DataFrame(flattened_image_array.reshape(1, -1), columns=['pixel{:04d}'.format(i) for i in range(28 * 28 * 3)])

        # 提取標籤
        if 'Benign keratosis' in filename:
            label = '2'
        elif 'Actinic keratosis' in filename:
            label = '0'
        elif 'Basal cell carcinoma' in filename:
            label = '1'
        elif 'Dermatofibroma' in filename:
            label = '3'
        elif 'Melanocytic nevus' in filename:
            label = '4'
        elif 'Melanoma' in filename:
            label = '6'
        elif 'Vascular_lesion' in filename:
            label = '5'
        elif 'normal' in filename:
            label = '7'

        # 添加標籤到DataFrame中
        df_temp['label'] = label

        # 將臨時DataFrame加到主DataFrame中
        df = pd.concat([df, df_temp], ignore_index=True)

# 保存DataFrame到 CSV 文件
df.to_csv('model_data.csv', index=False)

print("資料集轉換完成並保存為 CSV 檔案。")


資料集轉換完成並保存為 CSV 檔案。


In [29]:
import pandas as pd
data = pd.read_csv('model_data.csv')
data.head(100)

,pixel0000,pixel0001,pixel0002,pixel0003,pixel0004,pixel0005,pixel0006,pixel0007,pixel0008,pixel0009,...,pixel2343,pixel2344,pixel2345,pixel2346,pixel2347,pixel2348,pixel2349,pixel2350,pixel2351,label
0,180,138,116,177,135,113,182,137,114,183,...,198,155,136,197,154,135,196,153,134,2
1,232,155,125,222,147,116,222,149,117,217,...,209,112,96,207,111,95,206,110,94,1
2,232,195,177,220,183,165,209,170,155,205,...,181,123,109,179,122,105,178,121,104,5
3,247,159,109,247,161,110,251,165,118,253,...,193,109,65,189,106,62,187,104,60,6
4,189,154,116,188,153,115,189,154,116,192,...,163,128,88,166,131,93,166,131,93,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,248,191,161,247,190,160,253,192,161,252,...,253,182,150,253,180,147,251,178,145,2
96,185,148,142,185,148,142,186,149,143,190,...,146,119,110,145,118,109,144,117,108,1
97,98,62,46,123,88,69,120,83,64,126,...,117,74,57,109,66,49,107,66,48,6
98,249,215,180,245,211,176,255,223,188,248,...,237,193,164,234,193,163,233,192,162,2
